In [1]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
import geopandas as gpd

spark = SparkSession. \
    builder. \
    appName('Milestone3 - Airline Search'). \
    master('spark://DESKTOP-RV59F4C.localdomain:7077'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,'
           'org.datasyslab:geotools-wrapper:geotools-24.1'). \
    getOrCreate()

SedonaRegistrator.registerAll(spark)

21/11/11 11:56:35 WARN Utils: Your hostname, DESKTOP-RV59F4C resolves to a loopback address: 127.0.1.1; using 192.168.20.1 instead (on interface eth0)
21/11/11 11:56:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/root/spark-3.0.3-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-72e4b39c-f84c-4820-8e02-230810a68daf;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.0.1-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.j

True

## Load state boundries in WKT TSV and convert WKT string column to a geometry column

In [2]:
states_wkt = spark.read.option("delimiter", "\t").option("header", "false").csv("boundary-each-state.tsv").toDF("s_name","s_bound")
states_wkt.show()
states_wkt.printSchema()

states = states_wkt.selectExpr("s_name", "ST_GeomFromWKT(s_bound) as s_bound")
states.show()
states.printSchema()
states.createOrReplaceTempView("states")

+-------------+--------------------+
|       s_name|             s_bound|
+-------------+--------------------+
|       Alaska|POLYGON((-141.020...|
|      Alabama|POLYGON((-88.1955...|
|     Arkansas|POLYGON((-94.0416...|
|      Arizona|POLYGON((-112.598...|
|   California|POLYGON((-124.400...|
|     Colorado|POLYGON((-109.044...|
|  Connecticut|POLYGON((-73.4875...|
|     Delaware|POLYGON((-75.7919...|
|      Florida|POLYGON((-87.6050...|
|      Georgia|POLYGON((-85.6082...|
|       Hawaii|POLYGON((-154.628...|
|         Iowa|POLYGON((-95.7623...|
|        Idaho|POLYGON((-117.031...|
|     Illinois|POLYGON((-90.6290...|
|      Indiana|POLYGON((-87.5253...|
|       Kansas|POLYGON((-102.050...|
|     Kentucky|POLYGON((-89.5372...|
|    Louisiana|POLYGON((-94.0430...|
|Massachusetts|POLYGON((-72.7789...|
|     Maryland|POLYGON((-79.4778...|
+-------------+--------------------+
only showing top 20 rows

root
 |-- s_name: string (nullable = true)
 |-- s_bound: string (nullable = true)



+-------------+--------------------+
|       s_name|             s_bound|
+-------------+--------------------+
|       Alaska|POLYGON ((-141.02...|
|      Alabama|POLYGON ((-88.195...|
|     Arkansas|POLYGON ((-94.041...|
|      Arizona|POLYGON ((-112.59...|
|   California|POLYGON ((-124.40...|
|     Colorado|POLYGON ((-109.04...|
|  Connecticut|POLYGON ((-73.487...|
|     Delaware|POLYGON ((-75.791...|
|      Florida|POLYGON ((-87.605...|
|      Georgia|POLYGON ((-85.608...|
|       Hawaii|POLYGON ((-154.62...|
|         Iowa|POLYGON ((-95.762...|
|        Idaho|POLYGON ((-117.03...|
|     Illinois|POLYGON ((-90.629...|
|      Indiana|POLYGON ((-87.525...|
|       Kansas|POLYGON ((-102.05...|
|     Kentucky|POLYGON ((-89.537...|
|    Louisiana|POLYGON ((-94.043...|
|Massachusetts|POLYGON ((-72.778...|
|     Maryland|POLYGON ((-79.477...|
+-------------+--------------------+
only showing top 20 rows

root
 |-- s_name: string (nullable = true)
 |-- s_bound: geometry (nullable = false)



## Importing airports.dat and perform data cleaning

In [224]:
airport_dat = spark.read.option("delimiter", ",").option("header", "false").csv("airports.dat").toDF("AirportID","Name"
,"City","Country","IATA","ICAO","Latitude","Longitude","Altitude","Timezone","DST","Database Time Zone","Type","Source")
airport_dat = airport_dat.drop("Source")
airport_dat = airport_dat.drop("Type")
airport_dat = airport_dat.drop("Database Time Zone")
airport_dat = airport_dat.drop("DST")
airport_dat = airport_dat.drop("Timezone")
airport_dat = airport_dat.drop("Altitude")
airport_dat = airport_dat.fillna('N/A')
airport_dat = airport_dat.na.replace("\\N","N/A")
airport_dat = airport_dat.na.replace("-","N/A")

from pyspark.sql import functions
airport = airport_dat.select("AirportID", "Name", "City", "Country","IATA","ICAO",
                                 functions.concat_ws("_",airport_dat.Longitude,airport_dat.Latitude).alias("Location"))

airport = airport.selectExpr("*", "ST_PointFromText(Location, \'_\') as geospacial_Location")
airport = airport.drop("Location")
airport.createOrReplaceTempView("airport")
airport.show()
airport.printSchema()

+---------+--------------------+--------------+----------------+----+----+--------------------+
|AirportID|                Name|          City|         Country|IATA|ICAO| geospacial_Location|
+---------+--------------------+--------------+----------------+----+----+--------------------+
|        1|      Goroka Airport|        Goroka|Papua New Guinea| GKA|AYGA|POINT (145.391998...|
|        2|      Madang Airport|        Madang|Papua New Guinea| MAG|AYMD|POINT (145.789001...|
|        3|Mount Hagen Kagam...|   Mount Hagen|Papua New Guinea| HGU|AYMH|POINT (144.296005...|
|        4|      Nadzab Airport|        Nadzab|Papua New Guinea| LAE|AYNZ|POINT (146.725977...|
|        5|Port Moresby Jack...|  Port Moresby|Papua New Guinea| POM|AYPY|POINT (147.220001...|
|        6|Wewak Internation...|         Wewak|Papua New Guinea| WWK|AYWK|POINT (143.669006...|
|        7|  Narsarsuaq Airport|  Narssarssuaq|       Greenland| UAK|BGBW|POINT (-45.425998...|
|        8|Godthaab / Nuuk A...|      Go

## Importing airline.dat and perform data cleaning

In [4]:
airline_dat = spark.read.option("delimiter", ",").option("header", "false").csv("airlines.dat").toDF("AirlineID","Name"
,"Alias","IATA","ICAO","Callsign","Country","Active")
airline_dat = airline_dat.fillna('N/A')
airline_dat = airline_dat.na.replace("\\N","N/A")
airline_dat = airline_dat.na.replace("-","N/A")
airline_dat = airline_dat.drop("Callsign")
airline_dat.createOrReplaceTempView("airline")
airline_dat = spark.sql("select * from airline where AirlineID > 0")
airline_dat.show()
airline_dat.printSchema()
airline = airline_dat

+---------+--------------------+-----+----+----+--------------+------+
|AirlineID|                Name|Alias|IATA|ICAO|       Country|Active|
+---------+--------------------+-----+----+----+--------------+------+
|        1|      Private flight|  N/A| N/A| N/A|           N/A|     Y|
|        2|         135 Airways|  N/A| N/A| GNL| United States|     N|
|        3|       1Time Airline|  N/A|  1T| RNX|  South Africa|     Y|
|        4|2 Sqn No 1 Elemen...|  N/A| N/A| WYT|United Kingdom|     N|
|        5|     213 Flight Unit|  N/A| N/A| TFU|        Russia|     N|
|        6|223 Flight Unit S...|  N/A| N/A| CHD|        Russia|     N|
|        7|   224th Flight Unit|  N/A| N/A| TTF|        Russia|     N|
|        8|         247 Jet Ltd|  N/A| N/A| TWF|United Kingdom|     N|
|        9|         3D Aviation|  N/A| N/A| SEC| United States|     N|
|       10|         40-Mile Air|  N/A|  Q5| MLA| United States|     Y|
|       11|              4D Air|  N/A| N/A| QRT|      Thailand|     N|
|     

## Importing routes.dat

In [5]:
route_dat = spark.read.option("delimiter", ",").option("header", "false").csv("routes.dat").toDF("Airline","AirlineID"
,"SourceAirport","SourceAirportID","DestinationAirport","DestinationAirportID","Codeshare","Stops","Equipment")
route_dat = route_dat.drop("Equipment")
route_dat = route_dat.fillna('N/A')
route_dat = route_dat.na.replace("\\N","N/A")
route_dat = route_dat.na.replace("-","N/A")
route_dat.createOrReplaceTempView("route")
#route_dat.show()
#route_dat.printSchema()

route = route_dat
route.show()

+-------+---------+-------------+---------------+------------------+--------------------+---------+-----+
|Airline|AirlineID|SourceAirport|SourceAirportID|DestinationAirport|DestinationAirportID|Codeshare|Stops|
+-------+---------+-------------+---------------+------------------+--------------------+---------+-----+
|     2B|      410|          AER|           2965|               KZN|                2990|      N/A|    0|
|     2B|      410|          ASF|           2966|               KZN|                2990|      N/A|    0|
|     2B|      410|          ASF|           2966|               MRV|                2962|      N/A|    0|
|     2B|      410|          CEK|           2968|               KZN|                2990|      N/A|    0|
|     2B|      410|          CEK|           2968|               OVB|                4078|      N/A|    0|
|     2B|      410|          DME|           4029|               KZN|                2990|      N/A|    0|
|     2B|      410|          DME|           40

## Updating routes table

In [6]:
airport.count()

7698

In [7]:
airline.count()

6161

In [8]:
route.count()

67663

In [9]:
airline_cp = spark.sql("Select * from airline where airline.IATA != 'N/A'")
airline_cp = airline_cp.withColumnRenamed("IATA","Airline")

In [10]:
route_cp = route.alias('r').join(airline_cp.alias('a'), on = 'Airline', how = 'left')\
    .select('r.Airline', functions.when(~functions.isnull(functions.col('a.AirlineID')),
                                       functions.col('a.AirlineID')
                                       ).otherwise(functions.col('r.AirlineID')).alias('AirlineID'),
            #'r.AirlineID', 'a.AirlineID',
           'r.SourceAirport', 'r.SourceAirportID', 'r.DestinationAirport', 'r.DestinationAirportID',
           'r.Codeshare', 'r.Stops').dropDuplicates()

In [11]:
route_cp.count()

78221

In [12]:
airline_cp = spark.sql("Select * from airline where airline.ICAO != 'N/A'")
airline_cp = airline_cp.withColumnRenamed("ICAO","Airline")

In [13]:
route_cp = route_cp.alias('r').join(airline_cp.alias('a'), on = 'Airline', how = 'left')\
    .select('r.Airline', functions.when(~functions.isnull(functions.col('a.AirlineID')),
                                       functions.col('a.AirlineID')
                                       ).otherwise(functions.col('r.AirlineID')).alias('AirlineID'),
           'r.SourceAirport', 'r.SourceAirportID', 'r.DestinationAirport', 'r.DestinationAirportID',
           'r.Codeshare', 'r.Stops').dropDuplicates()

In [14]:
airport_cp = spark.sql("Select * from airport where airport.IATA != 'N/A'")
airport_cp = airport_cp.withColumnRenamed("IATA","SourceAirport")

In [15]:
route_cp = route_cp.alias('r').join(airport_cp.alias('a'), on = 'SourceAirport', how = 'left')\
    .select('r.Airline', 'r.AirlineID','r.SourceAirport', functions.when(~functions.isnull(functions.col('a.AirportID')),
                                       functions.col('a.AirportID')
                                       ).otherwise(functions.col('r.SourceAirportID')).alias('SourceAirportID'), 
            'r.DestinationAirport', 'r.DestinationAirportID',
           'r.Codeshare', 'r.Stops').dropDuplicates()

In [16]:
airport_cp = spark.sql("Select * from airport where airport.ICAO != 'N/A'")
airport_cp = airport_cp.withColumnRenamed("ICAO","SourceAirport")

In [17]:
route_cp = route_cp.alias('r').join(airport_cp.alias('a'), on = 'SourceAirport', how = 'left')\
    .select('r.Airline', 'r.AirlineID','r.SourceAirport', functions.when(~functions.isnull(functions.col('a.AirportID')),
                                       functions.col('a.AirportID')
                                       ).otherwise(functions.col('r.SourceAirportID')).alias('SourceAirportID'), 
            'r.DestinationAirport', 'r.DestinationAirportID',
           'r.Codeshare', 'r.Stops').dropDuplicates()

In [18]:
airport_cp = spark.sql("Select * from airport where airport.IATA != 'N/A'")
airport_cp = airport_cp.withColumnRenamed("IATA","DestinationAirport")

In [19]:
route_cp = route_cp.alias('r').join(airport_cp.alias('a'), on = 'DestinationAirport', how = 'left')\
    .select('r.Airline', 'r.AirlineID','r.SourceAirport', 'r.SourceAirportID',
            'r.DestinationAirport', functions.when(~functions.isnull(functions.col('a.AirportID')),
                                       functions.col('a.AirportID')
                                       ).otherwise(functions.col('r.DestinationAirportID')).alias('DestinationAirportID'),
           'r.Codeshare', 'r.Stops').dropDuplicates()

In [20]:
route_cp.count()

78221

In [21]:
airport_cp = spark.sql("Select * from airport where airport.ICAO != 'N/A'")
airport_cp = airport_cp.withColumnRenamed("ICAO","DestinationAirport")

In [22]:
route_cp = route_cp.alias('r').join(airport_cp.alias('a'), on = 'DestinationAirport', how = 'left')\
    .select('r.Airline', 'r.AirlineID','r.SourceAirport', 'r.SourceAirportID',
            'r.DestinationAirport', functions.when(~functions.isnull(functions.col('a.AirportID')),
                                       functions.col('a.AirportID')
                                       ).otherwise(functions.col('r.DestinationAirportID')).alias('DestinationAirportID'),
           'r.Codeshare', 'r.Stops').dropDuplicates().sort("Airline")

In [23]:
route_cp.show()

+-------+---------+-------------+---------------+------------------+--------------------+---------+-----+
|Airline|AirlineID|SourceAirport|SourceAirportID|DestinationAirport|DestinationAirportID|Codeshare|Stops|
+-------+---------+-------------+---------------+------------------+--------------------+---------+-----+
|     3K|     3021|          SIN|           3316|               HKG|                3077|      N/A|    0|
|     3O|     9818|          CMN|           1074|               LYS|                1335|      N/A|    0|
|     3U|     4608|          CTU|           3395|               HKT|                3179|      N/A|    0|
|     3U|     4608|          CTU|           3395|               MEL|                3339|      N/A|    0|
|     3U|     4608|          CTU|           3395|               XNN|                6366|      N/A|    0|
|     3U|     4608|          KWE|           3394|               LHW|                3378|      N/A|    0|
|     3U|     4608|          SYD|           33

## Find the list of airports operating in country X

In [100]:
airport_cp = spark.sql("Select AirportID, Name, City, Country from airport where airport.Country != 'N/A'")
X = input("Enter country's name: ")
airport_cp.filter(airport_cp.Country.rlike(f"(?i)^*{X}$")).show()

Enter country's name: VIetnam
+---------+--------------------+----------------+-------+
|AirportID|                Name|            City|Country|
+---------+--------------------+----------------+-------+
|     3196|Da Nang Internati...|          Danang|Vietnam|
|     3197|    Gia Lam Air Base|           Hanoi|Vietnam|
|     3198|        Kep Air Base|             Kep|Vietnam|
|     3199|Noi Bai Internati...|           Hanoi|Vietnam|
|     3200|  Nha Trang Air Base|        Nhatrang|Vietnam|
|     3201|     Phu Bai Airport|             Hue|Vietnam|
|     3204|Phu Quoc Internat...|         Phuquoc|Vietnam|
|     3205|Tan Son Nhat Inte...|Ho Chi Minh City|Vietnam|
|     4153| Lien Khuong Airport|           Dalat|Vietnam|
|     4154|    Dong Hoi Airport|        Dong Hoi|Vietnam|
|     4155|    Rach Gia Airport|        Rach Gia|Vietnam|
|     4156|      Cà Mau Airport|          Ca Mau|Vietnam|
|     4157|Chu Lai Internati...|         Chu Lai|Vietnam|
|     4158|    Dong Tac Airport|         T

## Find the list of airlines having X stops

In [62]:
from pyspark.sql.functions import when
route_cp.createOrReplaceTempView("route_cp")
X = input("Enter # stops: ")
newRoute = spark.sql(f"Select AirlineID, Stops from route_cp where route_cp.Stops == {X}")
airline_cp = spark.sql(f"Select AirlineID, Name, Alias from airline")
newRoute = newRoute.alias('r').join(airline_cp.alias('a'), on = 'AirlineID', how = 'inner')\
    .select('r.AirlineID', when(airline_cp.Name != 'N/A', airline_cp.Name)
            .otherwise("a.Alias").alias('Airline_Name'),
'r.Stops').dropDuplicates()
newRoute.filter(newRoute.Airline_Name != 'N/A').show()

Enter # stops: 1


+---------+--------------------+-----+
|AirlineID|        Airline_Name|Stops|
+---------+--------------------+-----+
|     1316|     AirTran Airways|    1|
|     1936|  Cubana de Aviación|    1|
|     4319|Scandinavian Airl...|    1|
|     1623|      Canadian North|    1|
|      330|          Air Canada|    1|
|     4547|  Southwest Airlines|    1|
+---------+--------------------+-----+



## List of airlines operating with codeshares

In [70]:
newRoute = spark.sql(f"Select AirlineID, Codeshare from route_cp where route_cp.Codeshare == 'Y' \
                     or route_cp.Codeshare == 'y'")
airline_cp = spark.sql(f"Select AirlineID, Name, Alias from airline")
airline_cp = newRoute.alias('r').join(airline_cp.alias('a'), on = 'AirlineID', how = 'inner')\
    .select('r.AirlineID', when(airline_cp.Name != 'N/A', airline_cp.Name)
            .otherwise("a.Alias").alias('Airline_Name'),
'r.Codeshare').dropDuplicates()
airline_cp.filter(airline_cp.Airline_Name != 'N/A').show()

+---------+--------------------+---------+
|AirlineID|        Airline_Name|Codeshare|
+---------+--------------------+---------+
|     5179|               Tarom|        Y|
|      502|Abu Dhabi Amiri F...|        Y|
|      324|  All Nippon Airways|        Y|
|     2855|              Indigo|        Y|
|     5416|             WestJet|        Y|
|     5347|Virgin Atlantic A...|        Y|
|     1889|       Copa Airlines|        Y|
|     2421|               Flybe|        Y|
|     4559|Swiss Internation...|        Y|
|       28|     Asiana Airlines|        Y|
|     5309|    Vietnam Airlines|        Y|
|     1942|      Cyprus Airways|        Y|
|      137|          Air France|        Y|
|      241|          Air Sahara|        Y|
|     3378|   Malaysia Airlines|        Y|
|     1908|          Corsairfly|        Y|
|     1317|         Air Transat|        Y|
|     4292|    Rwandair Express|        Y|
|      197| Azerbaijan Airlines|        Y|
|     2056|           Dragonair|        Y|
+---------+

## Find the list of active airlines in the United States - Airline aggregation 

In [74]:
airline_cp = spark.sql("""
                       Select AirlineID, Name, Alias, Country, Active 
                       from airline 
                       where Country == 'United States' and (Active == 'Y' or Active == 'y')
                       """)
airline_cp = airline_cp.select(airline_cp.AirlineID, when(airline_cp.Name != 'N/A', airline_cp.Name)
            .otherwise(airline_cp.Alias).alias('Airline_Name'), airline_cp.Country, airline_cp.Active)
airline_cp.filter(airline_cp.Airline_Name != 'N/A').show()

+---------+--------------------+-------------+------+
|AirlineID|        Airline_Name|      Country|Active|
+---------+--------------------+-------------+------+
|       10|         40-Mile Air|United States|     Y|
|       22|      Aloha Airlines|United States|     Y|
|       24|   American Airlines|United States|     Y|
|       35|       Allegiant Air|United States|     Y|
|      109|Alaska Central Ex...|United States|     Y|
|      149|  Air Cargo Carriers|United States|     Y|
|      210|Airlift Internati...|United States|     Y|
|      281|America West Airl...|United States|     Y|
|      282|       Air Wisconsin|United States|     Y|
|      287|Allegheny Commute...|United States|     Y|
|      295|        Air Sunshine|United States|     Y|
|      315|        ATA Airlines|United States|     Y|
|      397|           Arrow Air|United States|     Y|
|      452|Atlantic Southeas...|United States|     Y|
|      659|American Eagle Ai...|United States|     Y|
|      792|          Access 

## Find the country (or) territory has the highest number of Airports 

In [87]:
airport_cp = spark.sql("""
                select Country, count(Country) as Num_Airports
                from airport
                where country != 'N/A'
                group by Country
                order by Num_Airports desc
            """)
airport_cp.first()

Row(Country='United States', Num_Airports=1512)

## Top K cities with most incoming airlines 

In [91]:
airline_cp = spark.sql("""
                       Select AirlineID, DestinationAirportID 
                       from route_cp 
                       where AirlineID != 'N/A' and DestinationAirportID != 'N/A'
                       """)
airline_cp.createOrReplaceTempView("airline_cp")
airport_cp = spark.sql("""
                       Select AirportID, City
                       from airport 
                       where City != 'N/A'
                       """)
airport_cp.createOrReplaceTempView("airport_cp")
result = spark.sql("""
            Select airport_cp.City, count(distinct airline_cp.AirlineID) as Num_Airlines
            from airline_cp
            inner join airport_cp
            on airline_cp.DestinationAirportID == airport_cp.AirportID
            group by airport_cp.City
            order by Num_Airlines desc
    """)
result.show()

+---------+------------+
|     City|Num_Airlines|
+---------+------------+
|    Paris|         147|
|   London|         139|
|   Moscow|         132|
|  Bangkok|         126|
|Frankfurt|         111|
|     Rome|         103|
| Istanbul|          99|
|    Dubai|          97|
|Hong Kong|          97|
|Singapore|          94|
|Amsterdam|          93|
|   Munich|          93|
|  Beijing|          91|
| New York|          86|
|Barcelona|          85|
|   Milano|          85|
|   Zurich|          83|
|   Vienna|          83|
| Shanghai|          82|
|    Tokyo|          82|
+---------+------------+
only showing top 20 rows



## Find the closest airport to a city X’s geospatial coordinate 

In [227]:
X = input("Enter city's longitude: ")
Y = input("Enter country's latitude: ")

airport_cp = spark.sql("""
    select AirportID,Name,City,Country,geospacial_Location
    from airport
    where City != 'N/A'
""")
airport_cp.createOrReplaceTempView("airport_cp")
result = spark.sql(f" \
    select a.AirportID, a.Name, a.City, a.Country, ST_Distance(a.geospacial_Location, ST_Point({X}, {Y})) as Distance \
    from airport as a \
    order by Distance")
result.show()

Enter city's longitude: -122.313323
Enter country's latitude: 47.622715
+---------+--------------------+--------------+-------------+-------------------+
|AirportID|                Name|          City|      Country|           Distance|
+---------+--------------------+--------------+-------------+-------------------+
|     3726|Boeing Field King...|       Seattle|United States|0.09340483758251691|
|     8796|Renton Municipal ...|        Renton|United States|0.16208440203787616|
|     3577|Seattle Tacoma In...|       Seattle|United States|0.17376783195113774|
|     6455|Berkley Structure...|    Pagerungan|    Indonesia|0.17521683501593222|
|     3822|Snohomish County ...|       Everett|United States| 0.2853094707603653|
|     8446|Tacoma Narrows Ai...|        Tacoma|United States| 0.4426625752853818|
|     6445|Bremerton Nationa...|     Bremerton|United States| 0.4707141116100695|
|     3672|McChord Air Force...|        Tacoma|United States| 0.5115696708090709|
|     9348|    Enumclaw Ai

## Find the airport in each US state’s geospatial boundary 

In [225]:
result = spark.sql("""
    select s.s_name as States, a.AirportID, a.Name, a.City
    from states as s, airport as a
    where ST_Contains(s.s_bound, a.geospacial_Location)
""")
result.show()

21/11/11 16:43:37 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+-------------+---------+--------------------+----------------+
|       States|AirportID|                Name|            City|
+-------------+---------+--------------------+----------------+
|       Alaska|     3411|Barter Island LRR...|   Barter Island|
|       Alaska|     3412|Wainwright Air St...| Fort Wainwright|
|       Alaska|     3413|Cape Lisburne LRR...|   Cape Lisburne|
|       Alaska|     3414|Point Lay LRRS Ai...|       Point Lay|
|       Hawaii|     3415|Hilo Internationa...|            Hilo|
|      Florida|     3416|Orlando Executive...|         Orlando|
|       Alaska|     3417|     Bettles Airport|         Bettles|
|       Alaska|     3418|       Clear Airport|      Clear Mews|
|       Alaska|     3419|Indian Mountain L...|Indian Mountains|
|       Alaska|     3420|  Fort Yukon Airport|      Fort Yukon|
|       Alaska|     3421|Sparrevohn LRRS A...|      Sparrevohn|
|       Alaska|     3422|Bryant Army Heliport| Fort Richardson|
|       Alaska|     3423|Tatalina LRRS A